In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader
import pyvista as pv

In [2]:
# 기본적으로 plot이 안돼서 html로 plot하도록 만듦
pv.set_jupyter_backend('html')
pv.start_xvfb()

In [3]:
# Start an MAPDL instance
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)

In [4]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     13:07:57  NOV 27, 2024 CP=      1.104

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Kagome Lattice Creation

In [5]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
# density = 0.5

n_cell = 5
relative_density = 0.5

t_over_l = np.sqrt(3)/2 - np.sqrt(3 - 4*relative_density)/2
cell_size = design_height/(np.sqrt(3)*n_cell + t_over_l)
wall_thickness = cell_size*t_over_l

external_wall = True
external_wall_thickness = wall_thickness

In [6]:
side_length = cell_size - 1/np.sqrt(3)*wall_thickness
design_part = mapdl.rectng(-1/1000, design_width, 0, design_height)

In [7]:
for i in range(n_cell):
    j = 0
    while True:
        center = [cell_size/2 - np.sqrt(3)/2*wall_thickness + 2*j*cell_size - (i%2)*cell_size,
                  (cell_size + 1/np.sqrt(3)*wall_thickness)*np.sqrt(3)/2 + i*np.sqrt(3)*cell_size]
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if center[0] - side_length < design_width:
            k_list = []
            for k in range(6):
                k_list.append(mapdl.k(x = center[0] + side_length*np.cos(np.pi/3*k), y = center[1] + side_length*np.sin(np.pi/3*k)))
            hole = mapdl.a(*k_list)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                print(f"error2 at i={i}, j={j}")
        # 조건문 넣을 필요 없이, design part 늘리면 해결되긴 함
        if cell_size + 2*j*cell_size - (i%2)*cell_size < design_width:
            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = wall_thickness + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size - 1/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error2 at i={i}, j={j}")
                print(e)

            k1 = mapdl.k(x = cell_size + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k2 = mapdl.k(x = 2*cell_size - np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)*cell_size + i*np.sqrt(3)*cell_size)
            k3 = mapdl.k(x = (3/2)*cell_size - 1/2*np.sqrt(3)*wall_thickness + 2*j*cell_size - (i%2)*cell_size, y = np.sqrt(3)/2*cell_size + 3/2*wall_thickness + i*np.sqrt(3)*cell_size)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except Exception as e:
                print(f"error3 at i={i}, j={j}")
                print(e)

        if cell_size/2 - 1/np.sqrt(3)*wall_thickness + 2*j*cell_size > design_width:
            break

        j += 1

In [8]:
mapdl.rectng(x1=-total_width/2+design_width/2, x2=0, y1=0, y2=design_height)
mapdl.rectng(design_width, total_width/2+design_width/2, 0, design_height)
mapdl.aadd("all")

4

In [9]:
mapdl.vext("ALL", dz = total_z)

EXTRUDE ALL AREAS 
      IN DIRECTION   0.000000    ,  0.000000    , 0.5000000E-02
      SCALED         0.000000    ,  0.000000    ,  0.000000

In [ ]:
# line_nondesign_part = []
# for x in [(-total_width+design_width)/2, (total_width+design_width)/2]:
#     for y in [0, design_height]:
#         for z in [0, total_z]:
#             if x<0:
#                 k1 = mapdl.k(x=0, y=y, z=z)
#             else:
#                 k1 = mapdl.k(x=design_width, y=y, z=z)
#             k2 = mapdl.k(x=x, y=y, z=z)
#             line_nondesign_part.append(mapdl.l(k1, k2))

# for x in [(-total_width+design_width)/2, (total_width+design_width)/2]:
#     for z in [0, total_z]:
#         k1 = mapdl.k(x=x, y=0, z=z)
#         k2 = mapdl.k(x=x, y=design_height, z=z)
#         line_nondesign_part.append(mapdl.l(k1, k2))

## Preprocessing

In [10]:
stress_strain = pd.read_excel('PLA stress-strain.xlsx')
stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
stress_strain['y'] = stress_strain['y (MPa)']*1e6
stress_strain['x'] = stress_strain['x']/100
UTS = stress_strain['y'].max()
Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
max_plastic_strain = stress_strain['x'].iloc[-1]

mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

# Non linear 부분 정의
stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
mapdl.tb('PLASTIC', 1, '', 'MISO')
for i in range(len(stress_strain)):
    mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

In [11]:
line_design_part = mapdl.lsel("S", "LOC", "X", 0, design_width)
mapdl.lplot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [ ]:
# line_nondesign_part = np.ndarray([])

# for x in [(-total_width+design_width)/2, (total_width+design_width)/2]:
#     for y in [0, design_height]:
#         for z in [0, total_z]:
#             mapdl.lsel("S", "LOC", "Y", y)
#             if x > 0:
#                 mapdl.lsel("R", "LOC", "X", 0, (total_width+design_width)/2)
#             else:
#                 mapdl.lsel("R", "LOC", "X", (-total_width+design_width)/2, 0)
#             try:
#                 line_nondesign_part = np.concatenate((line_nondesign_part, mapdl.lsel("R", "LOC", "Z", z)))
#             except:
#                 line_nondesign_part = mapdl.lsel("R", "LOC", "Z", z)
# 
# # mapdl.lplot(cpos = 'xy')

In [ ]:
line_design_part = mapdl.lsel("S", "LOC", "X", 0, design_width)
mapdl.lsel("NONE")
for line in line_design_part:
    mapdl.lsel("A", "LINE", vmin=line, vmax=line)
mapdl.lesize("ALL", wall_thickness*2, kforc=1)
mapdl.lsel("ALL")

# Define mesh size
mapdl.esize(design_height/3)  # Default size

mapdl.mopt("EXPND", 0.7)  # Decrease the area mesh expansion.
mapdl.et(1, 'SOLID187')

# Mesh the volume
mapdl.vmesh("all")

GENERATE NODES AND ELEMENTS   IN  ALL  SELECTED VOLUMES  

 NUMBER OF VOLUMES MESHED   =         1
 MAXIMUM NODE NUMBER        =     37763
 MAXIMUM ELEMENT NUMBER     =     18903

In [13]:
mapdl.mesh.save("result/temp.vtk")

In [14]:
# grid = mapdl.mesh.grid
# grid.plot()

In [15]:
import pyvista as pv

# Use PyVista's notebook plotting
pv.set_jupyter_backend("html")  # For interactive plots
pv.start_xvfb()

# Read and display the VTK file
mesh = pv.read("result/temp.vtk")
mesh.plot(show_edges=True, color="grey", background="white")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [ ]:
# eplot보다 이게 더 빠름 ㅋㅋ
mapdl.solution()
mapdl.antype('STATIC')
mapdl.autots('OFF')
mapdl.nsubst(1, 1, 1)

mapdl.nsel('S', 'LOC', 'X', (total_width + design_width)/2)
mapdl.d('ALL', 'ALL', 0)
mapdl.nsel('S', 'LOC', 'X', (-total_width + design_width)/2)
mapdl.d("ALL", "ALL", 0)
mapdl.nsel('ALL')

mapdl.solve()
mapdl.post1()

result = mapdl.result
pyvista.set_jupyter_backend('html')
pyvista.start_xvfb()
result.plot_principal_nodal_stress(
    0,
    "SEQV",
    cpos="xy",
    background="w",
    text_color="k",
    add_text=True,
    show_edges=True,
)

## Simulation

In [ ]:
mapdl.parameters.keys()

In [19]:
a = [1, 3, 4]
b = [4, 5, 6]
pd.DataFrame({'a':a, 'b':b})

,a,b
0,1,4
1,3,5
2,4,6


## Saving 3D Object

In [1]:
import os

current_dir = os.getcwd()

In [19]:
mapdl.cdwrite(fname=f"{current_dir}/result/Kagome_3", ext = "txt")

*** MAPDL GLOBAL STATUS *** 

 TITLE =                                                                       
       504 KEYPOINTS DEFINED     MAX KEYPOINT NUMBER =            504
       756 LINES DEFINED         MAX LINE NUMBER =                756
       254 AREAS DEFINED         MAX AREA NUMBER =                254
         1 VOLUMES DEFINED       MAX VOLUME NUMBER =                1
         0 ELEM TYPES DEFINED    MAX ELEM TYPE NUMBER =             0
         0 ELEMENTS DEFINED      MAX ELEMENT NUMBER =               0
         0 NODES DEFINED         MAX NODE NUMBER =                  0

 WRITE ANSYS DATABASE AS AN ANSYS INPUT FILE: /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.txt                                                                                                                                                                                                                        

 WRITE IGES FILE= /home/mjk32/IIAD_pyMAPDL/result/Kagome_3.iges

 ATTRIBUTES WILL BE WRITTE

In [20]:
mapdl.exit()

In [4]:
result = pymapdl_reader.read_binary(f"{current_dir}/result/temp_triangular.rst")

FileNotFoundError: /home/mjk32/IIAD_pyMAPDL/result/temp_triangular.rst is not a file or cannot be found

## 2D Simulation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader
import pyvista as pv
import time
import traceback
import os

from functions import kagome_horizontal

In [9]:
# 기본적으로 plot이 안돼서 html로 plot하도록 만듦
pv.set_jupyter_backend('html')
pv.start_xvfb()

In [3]:
try:
    mapdl = launch_mapdl(start_instance=False, port=50052)
except:
    mapdl = launch_mapdl(start_instance=True, port=50052)


In [8]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Research     
 01078600  VERSION=LINUX x64     04:31:00  NOV 28, 2024 CP=     42.768

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [10]:
mapdl.clear()
mapdl.prep7()

# print(f"Simulation Start. Mesh size: {mesh_size}")
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
total_z = 5/1000
wall_thickness = kagome_horizontal(mapdl, n_cell=5, relative_density=0.5, three_dim=False)

In [11]:
# 물성치 입력
stress_strain = pd.read_excel('PLA stress-strain.xlsx')
stress_strain = stress_strain.iloc[:8]  # due to limitations of MISO model
stress_strain['y'] = stress_strain['y (MPa)']*1e6
stress_strain['x'] = stress_strain['x']/100
UTS = stress_strain['y'].max()
Youngs_Modulus = stress_strain['y'][1]/stress_strain['x'][1]
max_plastic_strain = stress_strain['x'].iloc[-1]

mapdl.mp('EX', 1, Youngs_Modulus)   # Young's Modulus in Pascals
mapdl.mp('PRXY', 1, 0.33)   # Poisson's Ratio

# Non linear 부분 정의
stress_strain['x'] = stress_strain['x'] - stress_strain['y']/Youngs_Modulus
mapdl.tb('PLASTIC', 1, '', 'MISO')
for i in range(len(stress_strain)):
    mapdl.tbpt(i, stress_strain['x'][i], stress_strain['y'][i])

In [12]:
# Meshing
line_design_part = mapdl.lsel("S", "LOC", "X", 0, design_width)
mapdl.lsel("NONE")
for line in line_design_part:
    mapdl.lsel("A", "LINE", vmin=line, vmax=line)
mapdl.lesize("ALL", wall_thickness/2, kforc=1)
mapdl.lsel("ALL")

# Define mesh size
mapdl.esize(design_height/6)  # Default size

mapdl.mopt("EXPND", 0.7)  # Decrease the area mesh expansion.
mapdl.et(1, 'PLANE')

# define a PLANE183 element type with thickness
mapdl.et(1, "PLANE183", kop1=1, kop3=3)
mapdl.r(1, total_z)  # thickness of 5mm

# Mesh the volume
mapdl.amesh("all")

GENERATE NODES AND ELEMENTS   IN  ALL  SELECTED AREAS    
    ** AREA     4 MESHED WITH        0 QUADRILATERALS,    14806 TRIANGLES **

 NUMBER OF AREAS MESHED     =          1
 MAXIMUM NODE NUMBER        =      32069
 MAXIMUM ELEMENT NUMBER     =      14806

In [13]:
mapdl.eplot()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…